### Dự đoán trạng thái, lỗi sai của động tác Lunge

In [2]:
# Thêm autoreload vào để tự động reload lại module nếu có thay đổi code trong module
%load_ext autoreload
%autoreload 2

import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
import warnings
warnings.filterwarnings('ignore')

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.common import load_model, calculate_angle

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Thực hiện việc dự đoán với các model Scikit learn có độ chính xác cao nhất

In [3]:
# Load model
# RF_model.pkl là sau điều chỉnh tham số
RF_model = load_model('./best_models/RF.pkl')

# Load input scaler
input_scaler = load_model("./best_models/input_scaler.pkl")

### Các landmarks quan trọng
![](../../images/landmarks_lunge.png)

In [4]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX"
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")

In [5]:
def extract_amd_recalculate_landmarks(pose_landmarks):
    """
    Tịnh tiến thân người vào giữa bức hình, đồng thời dời lại trục toạ độ
    """
    hip_center_x = float((pose_landmarks[23].x + pose_landmarks[24].x) / 2)
    hip_center_y = float((pose_landmarks[23].y + pose_landmarks[24].y) / 2)

    new_center = (0.5, 0.5)
    delta_x = new_center[0] - hip_center_x
    delta_y = new_center[1] - hip_center_y

    # Khởi tạo mảng NumPy với kích thước đã biết trước
    data = np.zeros((len(IMPORTANT_LMS), 3))

    for idx, landmark in enumerate(IMPORTANT_LMS):
        key_point_id = mp_pose.PoseLandmark[landmark].value
        key_point = pose_landmarks[key_point_id]
        data[idx] = [key_point.x + delta_x - 0.5, key_point.y + delta_y - 0.5, key_point.z]

    return data.flatten().tolist()


def extract_important_key_points(results) -> list:
    key_points = np.zeros((len(IMPORTANT_LMS), 3))  # Khởi tạo mảng NumPy

    for idx, lm in enumerate(IMPORTANT_LMS):
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[lm].value
        key_point = results.pose_landmarks.landmark[key_point_id]
        key_points[idx] = [key_point.x, key_point.y, key_point.z]

    return key_points.flatten().tolist()

def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [6]:
def get_class(encode_label: float):
    """
    Chuyển một label được encode thành class tương ứng
    """
    return {
        0: "Down",
        1: "Middle",
        2: "Stand"
    }.get(encode_label)

In [7]:
def draw_knee_angle(mp_results, image):
    image_shape = (image.shape[1], image.shape[0])
    landmarks = mp_results.pose_landmarks.landmark

    # Lấy toạ độ của 3 điểm cần thiết (hông, đầu gối, mắt cá chân) cho việc tính góc
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    # Tính góc tạo bởi 3 điểm trên ở bên trái và bên phải
    left_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_angle = calculate_angle(right_hip, right_knee, right_ankle)

    # Vẽ góc lên ảnh
    # trong đó 0.5 là font size, (255, 255, 255) là màu, 2 là độ dày của chữ
    cv2.putText(image, str(int(left_angle)), 
                tuple(np.multiply(left_knee, image_shape).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                )
    
    cv2.putText(image, str(int(right_angle)), 
                tuple(np.multiply(right_knee, image_shape).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                )

In [14]:
def check_errors(mp_results):
    leg_front = None # Kiểm tra chân nào là chân đang bước lên trước
    landmarks = mp_results.pose_landmarks.landmark
    if landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y > landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y:
        leg_front = "left"
    else:
        leg_front = "right"

    return leg_front
    # Kiểm tra xem gót chân của chân đưa lên có đang nhón lên không


In [9]:
VIDEO_TEST = "./example.mp4"

In [ ]:
import copy

cap = cv2.VideoCapture(VIDEO_TEST)
current_stage = "Unknown"
prediction_probability_threshold = 0.55

# Số frame được bỏ qua
frame_skip = 4
frame_count = 0

# Số rep tập được
counter = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            print("Ignoring empty camera frame.")
            break
        
        frame_count += 1
        
        # Bỏ qua frame nếu không phải frame được xử lý
        if frame_count % frame_skip != 0:
            continue

        # resize frame để tăng tốc độ xử lý
        image = rescale_frame(image, percent=30)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if not results.pose_landmarks:
            print("No human found")
            continue

        initial_pose_landmarks = copy.deepcopy(results.pose_landmarks)
        image.flags.writeable = True

        # Cần khôi phục lại màu gốc của ảnh
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))
        
        # Get landmarks
        try:
            draw_knee_angle(results, image)
            key_points = extract_amd_recalculate_landmarks(results.pose_landmarks.landmark)
            X = pd.DataFrame([key_points], columns=HEADERS[1:])
            X = input_scaler.transform(X)

            predicted_stage = RF_model.predict(X)[0]
            predicted_stage = get_class(predicted_stage)
            prediction_probability_max = RF_model.predict_proba(X)[0].max()

            if prediction_probability_max >= prediction_probability_threshold:
                if predicted_stage == "Down" and current_stage == "Middle":
                    counter += 1
                current_stage = predicted_stage

            cv2.rectangle(image, (0, 0), (image.shape[1], 60), (245, 117, 16), -1)

            # Display probability
            cv2.putText(image, "REP", (15, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display class
            cv2.putText(image, "STAGE", (100, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, f"{predicted_stage}, {round(prediction_probability_max, 2)}", (60, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, 
                        (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, "ERRORS", (260, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            errors = ""
            if current_stage == "Down":
                errors = check_errors(results)
            cv2.putText(image, f"{errors}", (260, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow("CV2", image)
        
        # Nhấn q để thoát
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()